In [1]:
%pwd

'd:\\Advanced Project\\time-estimation\\notebooks'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\Advanced Project\\time-estimation'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class DataIngestionEntity:
    root_dir: Path
    zip_file: Path

In [ ]:
from timeEstimator.Utils.common import create_directory, read_yaml
from timeEstimator.constant import *

In [9]:
class ConfigurationManager:
    def __init__(self, params=PARAMS_FILE_PATH, config=CONFIG_FILE_PATH):
        self.params = read_yaml(params)
        self.config = read_yaml(config)

        create_directory([self.config.root_dir])

    def data_ingestion_config(self):
        config = self.config.data_ingestion

        entity = DataIngestionEntity(
            root_dir = config.root_dir,
            zip_file = config.zip_file
        )

        return entity

In [10]:
from zipfile import ZipFile
from timeEstimator.logging import logger
from timeEstimator.Exception.exception import CustomException

In [11]:
class DataIngestion:
    def __init__(self, config : DataIngestionEntity):
        self.config = config

    def rename_folder(self):
        for file in os.listdir(self.config.root_dir):   
            if "raw" in file:
                os.rename(f"{self.config.root_dir}/{file}", f"{self.config.root_dir}/data_ingestion")
                logger.info(f"File Name Changed from {file} to data_ingestion")
                break

    def data_ingest(self):
        try:
            logger.info(f"Zip Extarction Started : {self.config.zip_file}")
            with ZipFile(self.config.zip_file, 'r') as f:
                f.extractall(self.config.root_dir)
                f.close()
            self.rename_folder()
            logger.info(f"Zip Extraction Finished at : {os.path.join(self.config.root_dir, "data_ingestion")}")
        except Exception as e:
            raise CustomException(str(e))

        

In [12]:
config = ConfigurationManager()
ingestion_config = config.data_ingestion_config()
data_ingestion = DataIngestion(config = ingestion_config)
data_ingestion.data_ingest()

[2025-02-25 21:27:48,481]: INFO: common : Read YAML File: params.yaml
[2025-02-25 21:27:48,483]: INFO: common : Read YAML File: config\config.yaml
[2025-02-25 21:27:48,484]: INFO: common : Directory has been Created: artifacts
[2025-02-25 21:27:48,485]: INFO: 1530169652 : Zip Extarction Started : raw.zip
[2025-02-25 21:28:15,262]: INFO: 1530169652 : File Name Changed from raw to data_ingestion
[2025-02-25 21:28:15,262]: INFO: 1530169652 : Zip Extraction Finished at : artifacts/data_ingestion
